# Timing Core Equity Factors

M2 203 Econometrics Project, Faune Blanchard

--- 

## Preliminary preparations

First and foremost, we start by importing the packages that we will need to conduct this work.

Then, we import the data file, and do some quick preprocessing.
We use Polars instead of Pandas because of the added speed and flexibility.

We also import locally from our pre-coded functions in models.py

In [1]:
import polars as pl 
import numpy as np

## **Part 1**  